In [61]:



%matplotlib inline
import csv
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import (MinMaxScaler, RobustScaler)
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.optimizers import Adam

from fancyimpute import KNN,  SoftImpute, IterativeImputer, BiScaler, MatrixFactorization
import util.data
rng = np.random.RandomState(42)


In [16]:
train_data, test_data = util.data.load()

Training Data: 
  Amount of features: 888
  Amount of observations: 1212
  Min age: 42.0 Max age: 96.0

Test Data: 
  Amount of observations: 776


# Use various ways to fill the nans

In [17]:
X_incomplete = train_data.drop(["id", "y"], axis=1).values
X = {}


k = 12
# Use nearest rows which have a feature to fill in each row's missing features
X["KNN"] = KNN(k=k).fit_transform(X_incomplete)


# Instead of solving the nuclear norm objective directly, instead
# induce sparsity using singular value thresholding
X_incomplete_normalized = BiScaler().fit_transform(X_incomplete)
X["SoftImpute"] = SoftImpute().fit_transform(X_incomplete_normalized)

X["MatrixFactorization"] = MatrixFactorization(learning_rate= 0.001, rank=50).fit_transform(X_incomplete)


train_mean_values = train_data.mean()
train_data_mean =  train_data.fillna(train_mean_values)
X["Mean"] = train_data_mean.drop(["id", "y"], axis=1).values






Imputing row 1/1212 with 66 missing, elapsed time: 7.865
Imputing row 101/1212 with 60 missing, elapsed time: 7.944
Imputing row 201/1212 with 68 missing, elapsed time: 8.027
Imputing row 301/1212 with 68 missing, elapsed time: 8.117
Imputing row 401/1212 with 63 missing, elapsed time: 8.201
Imputing row 501/1212 with 66 missing, elapsed time: 8.279
Imputing row 601/1212 with 77 missing, elapsed time: 8.364
Imputing row 701/1212 with 65 missing, elapsed time: 8.445
Imputing row 801/1212 with 61 missing, elapsed time: 8.529
Imputing row 901/1212 with 73 missing, elapsed time: 8.620
Imputing row 1001/1212 with 58 missing, elapsed time: 8.702
Imputing row 1101/1212 with 73 missing, elapsed time: 8.784
Imputing row 1201/1212 with 53 missing, elapsed time: 8.871
[BiScaler] Initial log residual value = 81.892772
[BiScaler] Iter 1: log residual = 1.651141, log improvement ratio=80.241632
[BiScaler] Iter 2: log residual = 0.995476, log improvement ratio=0.655665
[BiScaler] Iter 3: log residual

Epoch 31/10000
898130/898130 [==============================] - 1s 1us/step - loss: 1.0084 - val_loss: 1.0197
Epoch 32/10000
898130/898130 [==============================] - 1s 1us/step - loss: 1.0073 - val_loss: 1.0191
Epoch 33/10000
898130/898130 [==============================] - 1s 1us/step - loss: 1.0063 - val_loss: 1.0185
Epoch 34/10000
898130/898130 [==============================] - 1s 1us/step - loss: 1.0052 - val_loss: 1.0179
Epoch 35/10000
898130/898130 [==============================] - 1s 1us/step - loss: 1.0042 - val_loss: 1.0173
Epoch 36/10000
898130/898130 [==============================] - 1s 1us/step - loss: 1.0031 - val_loss: 1.0166
Epoch 37/10000
898130/898130 [==============================] - 1s 1us/step - loss: 1.0020 - val_loss: 1.0160
Epoch 38/10000
898130/898130 [==============================] - 1s 1us/step - loss: 1.0009 - val_loss: 1.0153
Epoch 39/10000
898130/898130 [==============================] - 1s 1us/step - loss: 0.9997 - val_loss: 1.0146
Epoch 40/1

Epoch 106/10000
898130/898130 [==============================] - 1s 1us/step - loss: 0.8980 - val_loss: 0.9396
Epoch 107/10000
898130/898130 [==============================] - 1s 1us/step - loss: 0.8967 - val_loss: 0.9388
Epoch 108/10000
898130/898130 [==============================] - 1s 1us/step - loss: 0.8954 - val_loss: 0.9380
Epoch 109/10000
898130/898130 [==============================] - 1s 1us/step - loss: 0.8942 - val_loss: 0.9372
Epoch 110/10000
898130/898130 [==============================] - 1s 1us/step - loss: 0.8929 - val_loss: 0.9365
Epoch 111/10000
898130/898130 [==============================] - 1s 1us/step - loss: 0.8917 - val_loss: 0.9357
Epoch 112/10000
898130/898130 [==============================] - 1s 1us/step - loss: 0.8905 - val_loss: 0.9350
Epoch 113/10000
898130/898130 [==============================] - 1s 1us/step - loss: 0.8893 - val_loss: 0.9343
Epoch 114/10000
898130/898130 [==============================] - 1s 1us/step - loss: 0.8881 - val_loss: 0.9336
E

In [37]:
Y = {}
y = train_data["y"].values
Y["KNN"] = y
Y["SoftImpute"] = y
Y["MatrixFactorization"] = y
Y["Mean"] = y


# Remove the outliers

In [26]:
# fit the model compute the otliers
outliers = {}
isoForest = IsolationForest(behaviour='new', max_samples=100, random_state=rng, contamination='auto')
for key in X:
    outliers[key] = isoForest.fit_predict(X[key])
print(outliers)


{'KNN': array([ 1, -1,  1, ...,  1,  1,  1]), 'SoftImpute': array([1, 1, 1, ..., 1, 1, 1]), 'MatrixFactorization': array([1, 1, 1, ..., 1, 1, 1]), 'Mean': array([1, 1, 1, ..., 1, 1, 1])}


In [38]:
# remove the outlier
for key in X:
    X[key] = X[key][np.where(outliers[key] > 0)]
    Y[key] = Y[key][np.where(outliers[key] > 0)]
    

# Scale the data 

In [41]:
for key in X:
    transformer = RobustScaler().fit(X[key])
    X[key] = transformer.transform(X[key])

# Compute the feature ranking for each imputer

In [43]:
# Define dictionary to store our rankings
ranks = {}
for key in X:
    ranks[key] = {}

# Create our function which stores the feature rankings to the ranks dictionary
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

In [45]:
colnames = train_data.drop(["id", "y"], axis=1).columns

rlasso = RandomizedLasso(alpha=0.04)
lr = LinearRegression(normalize=True)
ridge = Ridge(alpha = 7)
lasso = Lasso(alpha=.05)
rf = RandomForestRegressor(n_jobs=-1, n_estimators=50)


for key in X:
    print("Computing for {}".format(key))
    rlasso.fit(X[key], Y[key])
    ranks[key]["rlasso/Stability"] = ranking(np.abs(rlasso.scores_), colnames)
    print("rLasso done")
    # Construct our Linear Regression model

    lr.fit(X[key], Y[key])
    ranks[key]["LinReg"] = ranking(np.abs(lr.coef_), colnames)
    print("LinReg done")
    
    #stop the search when only the last feature is left
    rfe = RFE(lr, n_features_to_select=1 )
    rfe.fit(X[key], Y[key])
    ranks[key]["RFE"] = ranking(list(map(float, rfe.ranking_)), colnames, order=-1)
    print("RFE done")
    
    ridge.fit(X[key],Y[key])
    ranks[key]['Ridge'] = ranking(np.abs(ridge.coef_), colnames)
    print("Ridge done")
    
    lasso.fit(X[key], Y[key])
    ranks[key]["Lasso"] = ranking(np.abs(lasso.coef_), colnames)
    print("Lasso done")
    
    rf.fit(X[key],Y[key])
    ranks[key]["RF"] = ranking(rf.feature_importances_, colnames);
    print("RF done")
    
print("Done")

/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class RandomizedLasso is deprecated; The class RandomizedLasso is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


Computing for KNN


/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Computing for SoftImpute
Computing for MatrixFactorization


/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Computing for Mean


/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Done


In [46]:
# Create empty dictionary to store the mean value calculated from all the scores
r = {}
meanplot = {}
for key in X:
    r[key] = {}    
    for name in colnames:
        r[key][name] = round(np.mean([ranks[key][method][name] for method in ranks[key].keys()]), 2)
 
    methods = sorted(ranks[key].keys())
    ranks[key]["Mean"] = r[key]
    methods.append("Mean")
    
    # Put the mean scores into a Pandas dataframe
    meanplot[key] = pd.DataFrame(list(r[key].items()), columns= ['Feature','Mean Ranking'])

    # Sort the dataframe
    meanplot[key] = meanplot[key].sort_values('Mean Ranking', ascending=False)

In [60]:
# use the first 50 features
n_features_to_use = 50
combined_feature_index = set()
X_reduced_dim = {}
for key in meanplot:
    feature_list = list(meanplot[key].head(n_features_to_use)["Feature"].values) 
    combined_feature_index.update(feature_list)
    X_reduced_dim[key] = pd.DataFrame(X[key], columns=colnames)
    X_reduced_dim[key] = X_reduced_dim[key][feature_list]
    X_reduced_dim[key]["y"] = Y[key]
    X_reduced_dim[key].to_csv("preprocessed/" + key + ".csv", index=False)
    #X_reduced_dim[key] = X
    #train_subset = train_data_mean[feature_list + ["id", "y"]]
    


In [62]:
# create model
model = Sequential()
model.add(Dense(n_features_to_use, input_dim=50, init='RandomUniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(50, init='RandomUniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(100, init='RandomUniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(1, init='RandomUniform', activation='sigmoid'))
# Compile model
optimizer = Adam(lr=0.000005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='mean_absolute_error', optimizer=optimizer)



/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, input_dim=50, kernel_initializer="RandomUniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, kernel_initializer="RandomUniform")`
  """
/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, kernel_initializer="RandomUniform")`
  import sys
/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="RandomUniform")`
  if __name__ == '__main__':


# load test data and compute same preprocessin steps

In [64]:
X_test_incomplete = test_data.drop(["id"], axis=1).values
X_test = {}
X_test_reduced_dim = {}

k = 12
# Use nearest rows which have a feature to fill in each row's missing features
X_test["KNN"] = KNN(k=k).fit_transform(X_test_incomplete)
X_test["SoftImpute"] = SoftImpute().fit_transform(BiScaler().fit_transform(X_test_incomplete))
X_test["MatrixFactorization"] = MatrixFactorization(learning_rate= 0.001, rank=50).fit_transform(X_test_incomplete)
# use the training mean values
test_data_mean =  test_data.fillna(train_mean_values)
X_test["Mean"] = test_data_mean.drop(["id"], axis=1).values


Imputing row 1/776 with 63 missing, elapsed time: 3.032
Imputing row 101/776 with 54 missing, elapsed time: 3.100
Imputing row 201/776 with 63 missing, elapsed time: 3.167
Imputing row 301/776 with 54 missing, elapsed time: 3.241
Imputing row 401/776 with 65 missing, elapsed time: 3.326
Imputing row 501/776 with 53 missing, elapsed time: 3.406
Imputing row 601/776 with 71 missing, elapsed time: 3.491
Imputing row 701/776 with 70 missing, elapsed time: 3.559
[BiScaler] Initial log residual value = 81.909061
[BiScaler] Iter 1: log residual = 0.999783, log improvement ratio=80.909279
[BiScaler] Iter 2: log residual = -0.076421, log improvement ratio=1.076204
[BiScaler] Iter 3: log residual = -1.423854, log improvement ratio=1.347433
[BiScaler] Iter 4: log residual = -2.871885, log improvement ratio=1.448031
[BiScaler] Iter 5: log residual = -4.353028, log improvement ratio=1.481143
[BiScaler] Iter 6: log residual = -5.845094, log improvement ratio=1.492065
[BiScaler] Iter 7: log residual 

Epoch 33/10000
577737/577737 [==============================] - 0s 1us/step - loss: 1.0000 - val_loss: 1.0268
Epoch 34/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.9989 - val_loss: 1.0263
Epoch 35/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.9979 - val_loss: 1.0258
Epoch 36/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.9968 - val_loss: 1.0253
Epoch 37/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.9957 - val_loss: 1.0248
Epoch 38/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.9946 - val_loss: 1.0242
Epoch 39/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.9935 - val_loss: 1.0236
Epoch 40/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.9924 - val_loss: 1.0230
Epoch 41/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.9913 - val_loss: 1.0224
Epoch 42/1

Epoch 108/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.8861 - val_loss: 0.9485
Epoch 109/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.8847 - val_loss: 0.9477
Epoch 110/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.8834 - val_loss: 0.9469
Epoch 111/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.8820 - val_loss: 0.9461
Epoch 112/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.8807 - val_loss: 0.9453
Epoch 113/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.8794 - val_loss: 0.9446
Epoch 114/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.8781 - val_loss: 0.9439
Epoch 115/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.8769 - val_loss: 0.9431
Epoch 116/10000
577737/577737 [==============================] - 0s 1us/step - loss: 0.8756 - val_loss: 0.9425
E

In [69]:
# SCALE THE DATA
for key in X_test:
    transformer = RobustScaler().fit(X_test[key])
    X_test[key] = transformer.transform(X_test[key])

In [71]:
# SELECTED THE IMPORTANTANT FEATURES
for key in X_test:
    X_test_reduced_dim[key] = pd.DataFrame(X_test[key], columns=colnames)
    X_test_reduced_dim[key] = X_test_reduced_dim[key][feature_list]

In [72]:
predictions = {}
for key in X_reduced_dim:
    model.fit(x=X_reduced_dim[key].drop(["y"], axis=1).as_matrix(), 
              y=X_reduced_dim[key]["y"].values / 100.0, epochs=150, 
              verbose=1, 
              validation_split=0.1, 
              shuffle=True, 
              steps_per_epoch=100, initial_epoch=0, validation_steps=10)
    # calculate predictions
    
    predictions[key] = model.predict(X_test_reduced_dim[key].as_matrix()) * 100.0
    
     

/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 1088 samples, validate on 121 samples
Epoch 1/150
100/100 [==============================] - 1s 13ms/step - loss: 0.1990 - val_loss: 0.1964
Epoch 2/150
100/100 [==============================] - 1s 8ms/step - loss: 0.1985 - val_loss: 0.1958
Epoch 3/150
100/100 [==============================] - 1s 8ms/step - loss: 0.1979 - val_loss: 0.1951
Epoch 4/150
100/100 [==============================] - 1s 7ms/step - loss: 0.1972 - val_loss: 0.1943
Epoch 5/150
100/100 [==============================] - 1s 8ms/step - loss: 0.1963 - val_loss: 0.1933
Epoch 6/150
100/100 [==============================] - 1s 8ms/step - loss: 0.1952 - val_loss: 0.1921
Epoch 7/150
100/100 [==============================] - 1s 7ms/step - loss: 0.1939 - val_loss: 0.1907
Epoch 8/150
100/100 [==============================] - 1s 8ms/step - loss: 0.1923 - val_loss: 0.1889
Epoch 9/150
100/100 [==============================] - 1s 8ms/step - loss: 0.1904 - val_loss: 0.1867
Epoch 10/150
100/100 [=====================

Epoch 81/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0549 - val_loss: 0.0563
Epoch 82/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0546 - val_loss: 0.0562
Epoch 83/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0544 - val_loss: 0.0560
Epoch 84/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0542 - val_loss: 0.0559
Epoch 85/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0540 - val_loss: 0.0558
Epoch 86/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0538 - val_loss: 0.0557
Epoch 87/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0536 - val_loss: 0.0556
Epoch 88/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0534 - val_loss: 0.0555
Epoch 89/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0532 - val_loss: 0.0554
Epoch 90/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0530

100/100 [==============================] - 1s 7ms/step - loss: 0.0628 - val_loss: 0.0585
Epoch 11/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0621 - val_loss: 0.0578
Epoch 12/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0614 - val_loss: 0.0571
Epoch 13/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0608 - val_loss: 0.0565
Epoch 14/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0603 - val_loss: 0.0559
Epoch 15/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0597 - val_loss: 0.0554
Epoch 16/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0592 - val_loss: 0.0551
Epoch 17/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0588 - val_loss: 0.0547
Epoch 18/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0583 - val_loss: 0.0543
Epoch 19/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0579 - val_loss: 

100/100 [==============================] - 1s 8ms/step - loss: 0.0426 - val_loss: 0.0500
Epoch 91/150
100/100 [==============================] - 1s 9ms/step - loss: 0.0424 - val_loss: 0.0500
Epoch 92/150
100/100 [==============================] - 1s 9ms/step - loss: 0.0423 - val_loss: 0.0499
Epoch 93/150
100/100 [==============================] - 1s 9ms/step - loss: 0.0422 - val_loss: 0.0499
Epoch 94/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0421 - val_loss: 0.0499
Epoch 95/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0419 - val_loss: 0.0499
Epoch 96/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0418 - val_loss: 0.0499
Epoch 97/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0417 - val_loss: 0.0499
Epoch 98/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0415 - val_loss: 0.0499
Epoch 99/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0414 - val_loss: 

100/100 [==============================] - 1s 7ms/step - loss: 0.0556 - val_loss: 0.0567
Epoch 20/150
100/100 [==============================] - 1s 9ms/step - loss: 0.0552 - val_loss: 0.0564
Epoch 21/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0548 - val_loss: 0.0562
Epoch 22/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0544 - val_loss: 0.0559
Epoch 23/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0541 - val_loss: 0.0557
Epoch 24/150
100/100 [==============================] - 1s 10ms/step - loss: 0.0537 - val_loss: 0.0555
Epoch 25/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0534 - val_loss: 0.0554
Epoch 26/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0531 - val_loss: 0.0553
Epoch 27/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0528 - val_loss: 0.0552
Epoch 28/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0524 - val_loss:

100/100 [==============================] - 1s 7ms/step - loss: 0.0403 - val_loss: 0.0510
Epoch 100/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0402 - val_loss: 0.0510
Epoch 101/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0401 - val_loss: 0.0509
Epoch 102/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0400 - val_loss: 0.0509
Epoch 103/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0399 - val_loss: 0.0509
Epoch 104/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0397 - val_loss: 0.0509
Epoch 105/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0396 - val_loss: 0.0509
Epoch 106/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0395 - val_loss: 0.0509
Epoch 107/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0395 - val_loss: 0.0509
Epoch 108/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0394 - v

100/100 [==============================] - 1s 7ms/step - loss: 0.0525 - val_loss: 0.0541
Epoch 29/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0520 - val_loss: 0.0540
Epoch 30/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0515 - val_loss: 0.0538
Epoch 31/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0511 - val_loss: 0.0536
Epoch 32/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0507 - val_loss: 0.0535
Epoch 33/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0503 - val_loss: 0.0534
Epoch 34/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0499 - val_loss: 0.0533
Epoch 35/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0495 - val_loss: 0.0532
Epoch 36/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0492 - val_loss: 0.0531
Epoch 37/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0488 - val_loss: 

100/100 [==============================] - 1s 7ms/step - loss: 0.0354 - val_loss: 0.0515
Epoch 109/150
100/100 [==============================] - 1s 10ms/step - loss: 0.0353 - val_loss: 0.0515
Epoch 110/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0351 - val_loss: 0.0515
Epoch 111/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0350 - val_loss: 0.0516
Epoch 112/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0349 - val_loss: 0.0516
Epoch 113/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0348 - val_loss: 0.0517
Epoch 114/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0347 - val_loss: 0.0517
Epoch 115/150
100/100 [==============================] - 1s 7ms/step - loss: 0.0345 - val_loss: 0.0517
Epoch 116/150
100/100 [==============================] - 1s 8ms/step - loss: 0.0344 - val_loss: 0.0517
Epoch 117/150
100/100 [==============================] - 1s 12ms/step - loss: 0.0343 -

# Save predictions

In [89]:
for key in predictions:
    submission =  pd.DataFrame(data={"id": list(range(776)), "y": np.squeeze(list(predictions[key]))}) 
    submission.to_csv("submissions/{}_NN.csv".format(key), index= False)

In [73]:
X_subset = train_subset.drop(["id", "y"], axis=1).as_matrix()
Y_subset = train_subset["y"].values
rf = RandomForestRegressor(n_jobs=-1, n_estimators=50, verbose=3)
rf.fit(X_subset,Y_subset)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50building tree 44 of 5

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
           oob_score=False, random_state=None, verbose=3, warm_start=False)

In [86]:
# load the test set
test_set =  pd.read_csv(TEST_FILE_PATH)
test_set = target.fillna(train_mean_values)
test_set_sub = test_set[feature_list]

y_pred = rf.predict(test_set_sub.as_matrix())
test_set["y"] = y_pred



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


In [88]:
# create submission
test_set[["id", "y"]].to_csv("submissions/first.csv", index= False)